### GPU, cuda, gccの確認


In [1]:
!nvidia-smi
!nvcc --version
!gcc --version

Fri Dec  5 14:54:07 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.274.02             Driver Version: 535.274.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3070 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   54C    P8              16W /  90W |    466MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### パッケージのインストール / クリーンインストール
ssh先のjupyter環境などで実行の場合は, 当該環境にパッケージをインストール

In [ ]:
# クリーンが必要であれば実行
!rm -rf kineticEQ
#!rm -rf build

In [ ]:
# インストール
!git clone https://github.com/Minamium/kineticEQ.git
!pip install -e ./kineticEQ

### 動作確認

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# ここは自分のパッケージ構成に合わせて修正
# 例: kineticEQ/src/kineticEQ/bgk2d2v_core.py にあるなら
# from kineticEQ.bgk2d2v_core import BGK2D2V_core
from your_package_name.bgk2d2v_core import BGK2D2V_core


def main():
    # -----------------------------
    # 設定
    # -----------------------------
    device = "cuda" if torch.cuda.is_available() else "cpu"

    nx, ny = 64, 64
    nv_x, nv_y = 32, 32

    Lx, Ly = 1.0, 1.0
    vx_max, vy_max = 5.0, 5.0

    dt = 1e-3
    T_total = 0.1  # 100 ステップ

    tau_tilde = 0.1  # 適当な値。必要に応じて調整

    # -----------------------------
    # 初期モーメント場
    #   ここでは: 一様密度 n=1,
    #             温度 T=1,
    #             流速 u_x, u_y に弱いガウシアン攪乱を入れてみる
    # -----------------------------
    x = torch.linspace(0.0, Lx, nx)
    y = torch.linspace(0.0, Ly, ny)
    X, Y = torch.meshgrid(x, y, indexing="ij")

    # 一様密度・温度
    n0 = torch.ones((nx, ny))
    T0 = torch.ones((nx, ny))

    # ちょっとした渦状の u_x, u_y
    u_x0 = 0.1 * torch.sin(2.0 * np.pi * X) * torch.cos(2.0 * np.pi * Y)
    u_y0 = -0.1 * torch.cos(2.0 * np.pi * X) * torch.sin(2.0 * np.pi * Y)

    # -----------------------------
    # コアクラスのインスタンス生成
    # -----------------------------
    bgk = BGK2D2V_core(
        nx=nx,
        ny=ny,
        nv_x=nv_x,
        nv_y=nv_y,
        dt=dt,
        T_total=T_total,
        Lx=Lx,
        Ly=Ly,
        vx_max=vx_max,
        vy_max=vy_max,
        tau_tilde=tau_tilde,
        n=n0,
        u_x=u_x0,
        u_y=u_y0,
        T=T0,
        device=device,
        dtype="float64",
        use_tqdm=True,
        flag_record_state=True,   # アニメーション用に状態を記録
    )

    # -----------------------------
    # シミュレーション実行
    # -----------------------------
    bgk.run_simulation()

    # -----------------------------
    # animation_data から GIF を作成
    #    各ステップごとの n, u_x, u_y, T を 2x2 サブプロットで表示
    # -----------------------------
    states = bgk.animation_data
    if len(states) == 0:
        print("No animation data recorded. Set flag_record_state=True.")
        return

    fig, axes = plt.subplots(2, 2, figsize=(8, 6))
    ax_n, ax_ux, ax_uy, ax_T = axes[0, 0], axes[0, 1], axes[1, 0], axes[1, 1]

    # 初期フレーム
    frame0 = states[0]
    n_img = ax_n.imshow(frame0["n"], origin="lower", extent=[0, Lx, 0, Ly])
    ux_img = ax_ux.imshow(frame0["u_x"], origin="lower", extent=[0, Lx, 0, Ly])
    uy_img = ax_uy.imshow(frame0["u_y"], origin="lower", extent=[0, Lx, 0, Ly])
    T_img = ax_T.imshow(frame0["T"], origin="lower", extent=[0, Lx, 0, Ly])

    ax_n.set_title("Density n")
    ax_ux.set_title("u_x")
    ax_uy.set_title("u_y")
    ax_T.set_title("Temperature T")

    for ax in [ax_n, ax_ux, ax_uy, ax_T]:
        ax.set_xlabel("x")
        ax.set_ylabel("y")

    fig.tight_layout()

    # カラーバーを付けたければここで付けてもよい
    # fig.colorbar(n_img, ax=ax_n)
    # ...

    def update(frame_idx):
        state = states[frame_idx]
        n_img.set_data(state["n"])
        ux_img.set_data(state["u_x"])
        uy_img.set_data(state["u_y"])
        T_img.set_data(state["T"])

        fig.suptitle(f"t = {state['time']:.3f}")
        return [n_img, ux_img, uy_img, T_img]

    anim = animation.FuncAnimation(
        fig,
        update,
        frames=len(states),
        interval=100,  # ms
        blit=False,
    )

    # GIF として保存（PillowWriter 利用）
    gif_name = "bgk2d2v_moments.gif"
    print(f"Saving animation to {gif_name} ...")
    anim.save(gif_name, writer=animation.PillowWriter(fps=10))
    print("Done.")

    # 画面にも一応表示
    plt.show()


if __name__ == "__main__":
    main()
